In [2]:
import pandas as pd
from google.cloud import bigquery
import datetime

In [6]:
df_reviews1= pd.read_json('D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Florida/1.json', lines=True)
df_reviews2= pd.read_json('D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Florida/2.json', lines=True)
#df_reviews1= pd.read_json('C:/Data/GMaps/1.json', lines=True)
#df_reviews2= pd.read_json('C:/Data/GMaps/2.json', lines=True)

In [7]:
# se seleccionan los primeros 5 filas de 1.json y 2.json y se concatenan
df_reviews3= pd.concat([df_reviews1[:5],df_reviews2[:5]], ignore_index=True)

In [8]:
df_data= df_reviews3
df_data['time'] = pd.to_datetime(df_data['time'],unit='ms')     #Transforma la columna de tipo timestamp a formato datetime
df_data = df_data[(df_data['time'].dt.year >= 2010) & (df_data['time'].dt.year <= 2021)]
    
df_data= df_data.drop(['pics','resp','name', 'text'], axis=1)       #Elimina la columna pics
df_data= df_data.rename(columns={'time': 'date', 'gmap_id':'business_id', 'rating':'stars'})
df_data[['useful','funny','cool']] = 0
df_data['platform']= 1

df_data['user_id']= df_data['user_id'].astype(str)
df_data= df_data[['business_id','user_id', 'date', 'stars','useful','funny','cool','platform']]
df_reviews3= df_data

In [9]:
def check_reg_existe_en_bigQ(row, table_id, client):
    query = f"""
    SELECT COUNT(*)
    FROM `{table_id}`
    WHERE """
    
    conditions = []
    # ciclo para crear las condiciones del SQL
    for col, val in row.items():
        if isinstance(val, str):
            conditions.append(f"{col} = '{val}'")
        elif isinstance(val, (int, float)):
            conditions.append(f"{col} = {val}")
        elif isinstance(val, datetime.date):
            conditions.append(f"{col} = '{val.isoformat()}'")
        else:
            raise TypeError(f"Tipo de dato no soportado: {type(val)} en la columna {col}")
    
    query += " AND ".join(conditions)   #Une las condiciones al query principal
    print(query)
    query_job = client.query(query)
    result = query_job.result()     #Ejecuta el SQL
    
    for row in result:
        return row[0] > 0       #si el resultado es mayor a 0 devuelve TRUE

In [10]:
client = bigquery.Client()
project_id = 'proyectohenry2'
dataset_id = 'db_test'
table_id = 'reviews'
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

In [11]:
df_reviews3['existe_in_bigQ'] = df_reviews3.apply(lambda row: check_reg_existe_en_bigQ(row, full_table_id, client), axis=1)


    SELECT COUNT(*)
    FROM `proyectohenry2.db_test.reviews`
    WHERE business_id = '0x8893863ea87bd5dd:0x9383ebf973e74abb' AND user_id = '1.0147185615514873e+20' AND date = '2021-08-03T15:07:30.740000' AND stars = 1 AND useful = 0 AND funny = 0 AND cool = 0 AND platform = 1

    SELECT COUNT(*)
    FROM `proyectohenry2.db_test.reviews`
    WHERE business_id = '0x8893863ea87bd5dd:0x9383ebf973e74abb' AND user_id = '1.1547723478903833e+20' AND date = '2020-07-18T00:13:37.005000' AND stars = 5 AND useful = 0 AND funny = 0 AND cool = 0 AND platform = 1

    SELECT COUNT(*)
    FROM `proyectohenry2.db_test.reviews`
    WHERE business_id = '0x8893863ea87bd5dd:0x9383ebf973e74abb' AND user_id = '1.0180501024489284e+20' AND date = '2018-04-05T10:30:53.567000' AND stars = 5 AND useful = 0 AND funny = 0 AND cool = 0 AND platform = 1

    SELECT COUNT(*)
    FROM `proyectohenry2.db_test.reviews`
    WHERE business_id = '0x8893863ea87bd5dd:0x9383ebf973e74abb' AND user_id = '1.0634442288149375e+2

In [12]:
df_reviews3

,business_id,user_id,date,stars,useful,funny,cool,platform,existe_in_bigQ
0,0x8893863ea87bd5dd:0x9383ebf973e74abb,1.0147185615514873e+20,2021-08-03 15:07:30.740,1,0,0,0,1,True
1,0x8893863ea87bd5dd:0x9383ebf973e74abb,1.1547723478903833e+20,2020-07-18 00:13:37.005,5,0,0,0,1,True
2,0x8893863ea87bd5dd:0x9383ebf973e74abb,1.0180501024489284e+20,2018-04-05 10:30:53.567,5,0,0,0,1,True
3,0x8893863ea87bd5dd:0x9383ebf973e74abb,1.0634442288149375e+20,2016-07-07 16:10:19.586,1,0,0,0,1,True
4,0x8893863ea87bd5dd:0x9383ebf973e74abb,1.0087511306956177e+20,2016-12-02 12:56:55.081,5,0,0,0,1,True
5,0x88ecf453fa2e0e71:0x3e7cc80808a41906,1.0952903664414163e+20,2016-09-04 17:33:48.117,4,0,0,0,1,False
6,0x88ecf453fa2e0e71:0x3e7cc80808a41906,1.1521236785329376e+20,2019-11-08 06:41:24.103,1,0,0,0,1,False
7,0x88ecf453fa2e0e71:0x3e7cc80808a41906,1.15669795459122e+20,2020-07-23 19:56:53.620,5,0,0,0,1,False
8,0x88ecf453fa2e0e71:0x3e7cc80808a41906,1.0702660149386119e+20,2016-10-09 03:07:18.897,3,0,0,0,1,False
9,0x88ecf453fa2e0e71:0x3e7cc80808a41906,1.053953528013668e+20,2018-10-26 13:02:42.277,5,0,0,0,1,False
